<a name='0'></a>
## Импорт необходимых пакетов


In [1]:
import argparse
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import scipy.io
import scipy.misc
import numpy as np
import pandas as pd
import PIL
from PIL import ImageFont, ImageDraw, Image
import tensorflow as tf
from tensorflow.python.framework.ops import EagerTensor

from tensorflow.keras.models import load_model
from yad2k.models.keras_yolo import yolo_head
from yad2k.utils.utils import draw_boxes, get_colors_for_classes, scale_boxes, read_classes, read_anchors, preprocess_image

%matplotlib inline

<a name='1'></a>
## 1 - Постановка задачи

Для выполнения заданий дан видеоролик, содержащий видеоряд с автомобилями.

<center>
<video width="400" height="200" src="nb_images/road_video_compressed2.mp4" type="video/mp4" controls>
</video>
</center>

<caption><center> Видеоряд с автомобилями <br> Датасет взят с <a href="https://www.drive.ai/">drive.ai</a>.
</center></caption>

Все изображения из датасета были загружены в папку. Для каждого изображения были написаны массивы с метками, включающие в себя рамки для всех машин. Структура рамок (bounding box'ов, далее в тексте BB):
<caption><center> <img src="nb_images/box_label.png" style="width:500px;height:250;"></center></caption>

Если всего существует 80 классов, которые детектор объектов должен распознавать, можно представить метку класса $c$ либо как целое число от 1 до 80, либо как 80-мерный вектор (с 80 числами), один компонент которого равен 1, а остальные равны 0.

<a name='2'></a>
## 2 - YOLO

"You Only Look Once" (YOLO) — популярный алгоритм, поскольку он обеспечивает высокую точность и при этом может работать в режиме реального времени. Этот алгоритм «просматривает изображение только один раз» в том смысле, что для прогнозирования ему требуется только один проход прямого распространения по сети. После NMS он выводит распознанные объекты вместе с BB.

#### Входы и выходы модели
- **input** - набор изображений размером (608, 608, 3)
- **output** - список BB с принадлежностью к классам. В каждом BB содержится 6 позиций $(p_c, b_x, b_y, b_h, b_w, c)$. Если представить $c$ в виде вектора с 80 размерностями, то каждый BB содержит 85 чисел. 

#### Anchor Boxes
* Anchor boxes (далее AB) выбираются исходя из размеров объектов различных классов в датасете. Для выполнения заданий дается 5 AB, которые прописаны в файле './model_data/yolo_anchors.txt'
* Стуктура модели YOLO следующая: IMAGE (m, 608, 608, 3) -> DEEP CNN -> ENCODING (m, 19, 19, 5, 85).  


#### Энкодер

Если центр объекта попадает в ячейку сетки, эта ячейка отвечает за обнаружение этого объекта.

<caption><center> <img src="nb_images/architecture.png" style="width:700px;height:400;"> </center></caption>
<caption><center> <u> Энкеодер в YOLO </center></caption>




Так как используется 5 AB, каждая из 19 x 19 ячеек кодирует информацию о 5 AB. AB определяются только своей шириной и высотой.

Для упрощения можно развернуть две последних размерности (19, 19, 5, 85), тогда получим выход нейронной сети с размерностью (19, 19, 425).

<caption><center> <img src="nb_images/flatten.png" style="width:700px;height:400;"> </center></caption>
<caption><center>  Выход неронной сети  </center></caption>

#### Class score (CS)

Для каждой ячейки необходимо вычислить следующее  произведение (поэлементное) и найти вероятность того, что ячейка содержит определенный класс.  
CS находится как $score_{c,i} = p_{c} \times c_{i}$: вероятность того, что в ячейке есть объект $p_{c}$, умножается на вероятность того, что объект относится к определенному классу $c_{ я}$.


<caption><center> <img src="nb_images/probability_extraction.png" style="width:700px;height:400;"> </center></caption>
<caption><center> Определение класса для каждого BB </center></caption>

##### Пример

* Пусть для ячейки 1, вероятность существования объекта равна $p_{1}=0,60$.   Таким образом, вероятность того, что объект существует в ячейке 1, составляет 60%.  
* Вероятность того, что объект относится к классу 3 (автомобиль), равна $c_{3}=0,73$.  
* CS для поля 1 и категории «3» составляет $score_{1,3}=0,60 \times 0,73 = 0,44$.  
* Допустим, вы подсчитали CS для всех 80 классов в поле 1 и обнаружили, что балл для класса автомобиля является наибольшим. Таким образом, вы присвоите этой ячейке «1» CS 0,44 и класс «3».

#### Визуализация BB
Another way to visualize YOLO's output is to plot the bounding boxes that it outputs. Doing that results in a visualization like this:  

Один из способов визуализировать выходные данные YOLO — построить рамки, ограничивающие объекты. В результате можно увидеть следующую картину:

<caption><center> <img src="nb_images/anchor_map.png" style="width:200px;height:200;"> </center></caption>
<caption><center> Визуализация работы YOLO </center></caption>

#### Non-Max suppression (NMS) 

На рисунке выше показаны только те рамки, для которых значение вероятности присутствия объекта высоко, однако количество рамок все равно избыточно. 

Для уменьшения их количества используется алгоритм **non-max suppression**. Он состоит из следующих шагов:
- Откинуть рамки с низким CS. Это означает, что отбрасываются рамки, для которых нейросеть либо не уверена в существовании объекта, либо не уверена в принадлежности объекта к определенному классу. 
- Выбрать только одну рамку, когда несколько рамок перекрываются друг с другом и обнаруживают один и тот же объект.

### Фильтрация по пороговому значению

Сначала вы примените фильтр по пороговому значению, то есть вы откинете ячейки, для которых CS меньше выбранного порогового значения. 

На выходе модели генерируются 19x19x5x85 чисел, при этом каждое поле описывается 85 числами. Удобно разделить выходной тензор (19,19,5,85) (или (19,19,425)) на следующие переменные:

- `box_confidence`: тензор размером $(19, 19, 5, 1)$, содержащий $p_c$ для каждого AB (всего их 5) для каждой из 19x19 ячеек. 
- `boxes`: тензор размером $(19, 19, 5, 4)$, содержащий центр и размеры $(b_x, b_y, b_h, b_w)$ для каждого AB для каждой ячейки.
- `box_class_probs`: тензор размером $(19, 19, 5, 80)$, содержащий вероятности принадлежности объекта к каждому из классов $(c_1, c_2, ... c_{80})$ для каждого из AB для каждой из 19x19 ячеек.


Напишите функцию `yolo_filter_boxes()`.
1. Вычислите CS ($p \times c$).  

Подсказка по работе умножения массивов с разными размерностями: 
```python
a = np.random.randn(19, 19, 5, 1)
b = np.random.randn(19, 19, 5, 80)
c = a * b # размерность c будет равна (19, 19, 5, 80)
```

1. Для каждого AB найдите:
    - индекс класса с максимальным CS
    - соответствующий CS
    
    **Полезные ссылки**
        * [tf.math.argmax](https://www.tensorflow.org/api_docs/python/tf/math/argmax)
        * [tf.math.reduce_max](https://www.tensorflow.org/api_docs/python/tf/math/reduce_max)

2. Создайте маску по пороговому значению. Пример: `([0.9, 0.3, 0.4, 0.5, 0.1] < 0.4)` возвращает: `[False, True, False, False, True]`. В маске для AB, которые мы хотим сохранить, значение должно быть установлено равным `True`. 
3. Используйте функции TensorFlow для применения маски на `box_class_scores`, `boxes` и `box_classes`.

    **Полезные ссылки**:
    * [tf.boolean mask](https://www.tensorflow.org/api_docs/python/tf/boolean_mask)  

In [2]:
def yolo_filter_boxes(boxes, box_confidence, box_class_probs, threshold = .6):
    """    
    Входные аргументы:
        boxes -- тензор размером (19, 19, 5, 4)
        box_confidence -- тензор размером (19, 19, 5, 1)
        box_class_probs -- тензор размером (19, 19, 5, 80)
        threshold -- пороговое значение

    Выходные значения:
        scores -- тензор размером (None,), содержащий CS для данного AB 
        boxes -- тензор размером (None, 4), содержащий (b_x, b_y, b_h, b_w) координаты AB
        classes -- тензор размером (None,), содержащий индексы классов, присутствующих на AB

    Примечание: "None" используется т.к. мы не знаем заранее количество AB, прошедших фильтрацию по пороговому значению.
    """
    
    # Шаг 1: Вычисление CS (для конкретного box'a он далее называется box_score)

    box_scores = box_confidence * box_class_probs

    # Шаг 2: Поиск наиболее вероятного класса (исходя из box_classes), а также его CS.

    # Важно: установите параметр axis равным -1
    box_classes = tf.math.argmax(box_scores, axis=-1)
    box_class_scores = tf.math.reduce_max(box_scores, axis=-1)
    
    # Шаг 3: Создайте маску по пороговому значению
    filtering_mask = box_class_scores >= threshold
    
    # Шаг 4: Примените маску к box_class_scores, boxes и box_classes

    scores = tf.boolean_mask(box_class_scores, filtering_mask)
    boxes = tf.boolean_mask(boxes, filtering_mask)
    classes = tf.boolean_mask(box_classes, filtering_mask)
    
    return scores, boxes, classes

In [3]:
# Тестирование функции
tf.random.set_seed(10)
box_confidence = tf.random.normal([19, 19, 5, 1], mean=1, stddev=4, seed = 1)
boxes = tf.random.normal([19, 19, 5, 4], mean=1, stddev=4, seed = 1)
box_class_probs = tf.random.normal([19, 19, 5, 80], mean=1, stddev=4, seed = 1)
scores, boxes, classes = yolo_filter_boxes(boxes, box_confidence, box_class_probs, threshold = 0.5)
print("scores[2] = " + str(scores[2].numpy()))
print("boxes[2] = " + str(boxes[2].numpy()))
print("classes[2] = " + str(classes[2].numpy()))
print("scores.shape = " + str(scores.shape))
print("boxes.shape = " + str(boxes.shape))
print("classes.shape = " + str(classes.shape))

assert type(scores) == EagerTensor, "Use tensorflow functions"
assert type(boxes) == EagerTensor, "Use tensorflow functions"
assert type(classes) == EagerTensor, "Use tensorflow functions"

assert scores.shape == (1789,), "Wrong shape in scores"
assert boxes.shape == (1789, 4), "Wrong shape in boxes"
assert classes.shape == (1789,), "Wrong shape in classes"

assert np.isclose(scores[2].numpy(), 9.270486), "Values are wrong on scores"
assert np.allclose(boxes[2].numpy(), [4.6399336, 3.2303846, 4.431282, -2.202031]), "Values are wrong on boxes"
assert classes[2].numpy() == 8, "Values are wrong on classes"

print("\033[92m All tests passed!")
# END UNIT TEST

scores[2] = 9.270486
boxes[2] = [ 4.639934   3.2303846  4.431282  -2.2020311]
classes[2] = 8
scores.shape = (1789,)
boxes.shape = (1789, 4)
classes.shape = (1789,)
 All tests passed!


**Expected Output**:

<table>
    <tr>
        <td>
            <b>scores[2]</b>
        </td>
        <td>
           9.270486
        </td>
    </tr>
    <tr>
        <td>
            <b>boxes[2]</b>
        </td>
        <td>
           [ 4.6399336  3.2303846  4.431282  -2.202031 ]
        </td>
    </tr>
    <tr>
        <td>
            <b>classes[2]</b>
        </td>
        <td>
           8
        </td>
    </tr>
        <tr>
        <td>
            <b>scores.shape</b>
        </td>
        <td>
           (1789,)
        </td>
    </tr>
    <tr>
        <td>
            <b>boxes.shape</b>
        </td>
        <td>
           (1789, 4)
        </td>
    </tr>
    <tr>
        <td>
            <b>classes.shape</b>
        </td>
        <td>
           (1789,)
        </td>
    </tr>

</table>

### Non-max Suppression

Даже после фильтрации по пороговым значениям CS вы все равно получите множество перекрывающихся рамок. Вторым фильтром для выбора правильных рамок является NMS.


<caption><center>  <img src="nb_images/non-max-suppression.png" style="width:500px;height:400;"> </center></caption>
<caption><center> Пример использования NMS </center></caption>


NMS использует важную функцию с названием **"Intersection over Union"**, или IoU.

<caption><center>  <img src="nb_images/iou.png" style="width:500px;height:400;"> </center></caption>
<caption><center> Intersection over Union </center></caption>


Напишите функцию `iou()` 

Подсказки:
- В этом коде используется соглашение, согласно которому (0,0) — это верхний левый угол изображения, (1,0) — верхний правый угол и (1,1) — нижний правый угол. Другими словами, начало координат (0,0) - это верхний левый угол изображения. По мере увеличения x вы двигаетесь вправо. По мере увеличения y вы двигаетесь вниз.
- Здесь прямоугольник определяется с использованием двух его углов: верхнего левого $(x_1, y_1)$ и нижнего правого $(x_2,y_2)$ вместо использования средней точки, высоты и ширины. Это немного облегчает расчет пересечения.
- Чтобы вычислить площадь прямоугольника, умножьте его высоту $(y_2 - y_1)$ на ширину $(x_2 - x_1)$. 
  
- Чтобы найти **пересечение** двух прямоугольников $(xi_{1}, yi_{1}, xi_{2}, yi_{2})$: 
     - Верхний левый угол пересечения $(xi_{1}, yi_{1})$ находится путем сравнения верхних левых углов $(x_1, y_1)$ двух прямоугольников и поиска вершины с координатой x, которая правее и координаты y, которая ниже.
     - Правый нижний угол пересечения $(xi_{2}, yi_{2})$ находится путем сравнения нижних правых углов $(x_2,y_2)$ двух прямоугольников и поиска вершины, координата x которой левее, и координату y, которая выше.
     - Два прямоугольника **могут не пересекаться**.   Вы можете обнаружить это, если вычисленные вами координаты пересечения оказываются в верхнем правом и/или нижнем левом углах рамки пересечения. С другой стороны, если вы вычислите высоту $(y_2 - y_1)$ или ширину $(x_2 - x_1)$ и обнаружите, что хотя бы одна из этих длин отрицательна, то пересечения нет (площадь пересечения равна нулю).  
     - Два прямоугольника могут пересекаться по **краям или вершинам**, и в этом случае площадь пересечения по-прежнему равна нулю.   Это происходит, когда высота или ширина (или и то, и другое) рассчитанного пересечения равна нулю.

**Подсказки**

- `xi1` = максимальное значение координат x1 из двух прямоугольников 
- `yi1` = максимальное значение координат y1 из двух прямоугольников 
- `xi2` = максимальное значение координат x2 из двух прямоугольников 
- `yi2` = максимальное значение координат y2 из двух прямоугольников 
- `inter_area`: Можно использовать `max(height, 0)` и `max(width, 0)`


In [4]:
def iou(box1, box2):
    """
    Входные аргументы:
    box1 -- первый прямоугольник, список объектов с координатами (box1_x1, box1_y1, box1_x2, box_1_y2)
    box2 -- второй прямоугольник,  список объектов с координатами (box2_x1, box2_y1, box2_x2, box2_y2)
    """


    (box1_x1, box1_y1, box1_x2, box1_y2) = box1
    (box2_x1, box2_y1, box2_x2, box2_y2) = box2

    # Вычисление координат (yi1, xi1, yi2, xi2) пересечения  box1 и box2. Вычисление площади пересечения.
    xi1 = max(box1_x1, box2_x1)
    yi1 = max(box1_y1, box2_y1)
    xi2 = min(box1_x2, box2_x2)
    yi2 = min(box1_y2, box2_y2)
    inter_width = xi2 - xi1
    inter_height = yi2 - yi1 
    inter_area = max(inter_width, 0) * max(inter_height, 0)
    
    # Вычисление общей площади по формуле Union(A,B) = A + B - Inter(A,B)
    
    box1_area = (box1_x2 - box1_x1) * (box1_y2 - box1_y1)
    box2_area = (box2_x2 - box2_x1) * (box2_y2 - box2_y1)
    union_area = box1_area + box2_area - inter_area
    
    # Вычисление IoU
    iou = inter_area / union_area
    
    return iou

In [5]:
## Тест 1: пересечение прямоугольников
box1 = (2, 1, 4, 3)
box2 = (1, 2, 3, 4)

print("iou for intersecting boxes = " + str(iou(box1, box2)))
assert iou(box1, box2) < 1, "The intersection area must be always smaller or equal than the union area."
assert np.isclose(iou(box1, box2), 0.14285714), "Wrong value. Check your implementation. Problem with intersecting boxes"

## Тест 2: прямоугольники не пересекаются
box1 = (1,2,3,4)
box2 = (5,6,7,8)
print("iou for non-intersecting boxes = " + str(iou(box1,box2)))
assert iou(box1, box2) == 0, "Intersection must be 0"

## Тест 3: прямоугольники пересекаются по вершинам
box1 = (1,1,2,2)
box2 = (2,2,3,3)
print("iou for boxes that only touch at vertices = " + str(iou(box1,box2)))
assert iou(box1, box2) == 0, "Intersection at vertices must be 0"

## Тест 4: прямоугольники пересекаются по краям
box1 = (1,1,3,3)
box2 = (2,3,3,4)
print("iou for boxes that only touch at edges = " + str(iou(box1,box2)))
assert iou(box1, box2) == 0, "Intersection at edges must be 0"

print("\033[92m All tests passed!")


iou for intersecting boxes = 0.14285714285714285
iou for non-intersecting boxes = 0.0
iou for boxes that only touch at vertices = 0.0
iou for boxes that only touch at edges = 0.0
 All tests passed!


**Ожидаемый вывод**:

```
iou for intersecting boxes = 0.14285714285714285
iou for non-intersecting boxes = 0.0
iou for boxes that only touch at vertices = 0.0
iou for boxes that only touch at edges = 0.0
```


###  YOLO NMS

You are now ready to implement non-max suppression. The key steps are: 
1. Select the box that has the highest score.
2. Compute the overlap of this box with all other boxes, and remove boxes that overlap significantly (iou >= `iou_threshold`).
3. Go back to step 1 and iterate until there are no more boxes with a lower score than the currently selected box.

Теперь вы готовы реализовать NMS. Ключевые шаги: 
1. Выберите AB с наибольшим CS.
2. Вычислите перекрытие этого AB со всеми AB и удалите те, которые значительно перекрываются (iou >= `iou_threshold`).
3. Вернитесь к шагу 1 и повторяйте действия до тех пор, пока не останется AB с более низким CS, чем у текущего выбранного AB.

Это удалит все AB, которые сильно перекрываются с выбранными AB. Остаются только «лучшие» AB.


Напишите функцию `yolo_non_max_suppression()` используюя TensorFlow. TensorFlow имеет встроенные функции, которые помогают реализовать NMS:

**Документация**: 

- [tf.image.non_max_suppression()](https://www.tensorflow.org/api_docs/python/tf/image/non_max_suppression)
```
tf.image.non_max_suppression(
    boxes,
    scores,
    max_output_size,
    iou_threshold=0.5,
    name=None
)
```

- [tf.gather()](https://www.tensorflow.org/api_docs/python/tf/gather)
```
keras.gather(
    reference,
    indices
)
```

In [6]:
def yolo_non_max_suppression(scores, boxes, classes, max_boxes = 10, iou_threshold = 0.5):
    """
    
    Входные аргументы:
    scores -- тензор с размерностью (None,), выход функции of yolo_filter_boxes()
    boxes -- тензор с размерностью (None, 4), output функции yolo_filter_boxes(), отмасштабированный на размер изображения  (см. далее)
    classes -- тензор с размерностью (None,), output функции yolo_filter_boxes()
    max_boxes -- целое число, максимальное количество предсказанных рамок на выходе алгоритма
    iou_threshold -- пороговое значение "intersection over union"для алгоритма NMS
    
    Выходные значения:
    scores -- тензор с размерностью (None, ), предсказанный CS для каждой рамки (AB)
    boxes -- тензор с размерностью (None, 4), предсказанные координаты рамки
    classes -- тензор с размерностью (None, ),предсказанный класс для каждой рамки
    
    """
    
    # Используйте tf.image.non_max_suppression() чтобы получить список индексов, соответствующих рамкам, которые нужно оставить

    nms_indices = tf.image.non_max_suppression(boxes, scores, max_output_size=max_boxes, iou_threshold=iou_threshold)
    
    # Используйте tf.gather(), чтобы выбрать только nms_indices из scores, boxes и classes
    scores = tf.gather(scores, nms_indices)
    boxes = tf.gather(boxes, nms_indices)
    classes = tf.gather(classes, nms_indices)

    
    return scores, boxes, classes

In [7]:
# BEGIN UNIT TEST
tf.random.set_seed(10)
scores = tf.random.normal([54,], mean=1, stddev=4, seed = 1)
boxes = tf.random.normal([54, 4], mean=1, stddev=4, seed = 1)
classes = tf.random.normal([54,], mean=1, stddev=4, seed = 1)
scores, boxes, classes = yolo_non_max_suppression(scores, boxes, classes)

assert type(scores) == EagerTensor, "Use tensoflow functions"
print("scores[2] = " + str(scores[2].numpy()))
print("boxes[2] = " + str(boxes[2].numpy()))
print("classes[2] = " + str(classes[2].numpy()))
print("scores.shape = " + str(scores.numpy().shape))
print("boxes.shape = " + str(boxes.numpy().shape))
print("classes.shape = " + str(classes.numpy().shape))

assert type(scores) == EagerTensor, "Use tensoflow functions"
assert type(boxes) == EagerTensor, "Use tensoflow functions"
assert type(classes) == EagerTensor, "Use tensoflow functions"

assert scores.shape == (10,), "Wrong shape"
assert boxes.shape == (10, 4), "Wrong shape"
assert classes.shape == (10,), "Wrong shape"

assert np.isclose(scores[2].numpy(), 8.147684), "Wrong value on scores"
assert np.allclose(boxes[2].numpy(), [ 6.0797963, 3.743308, 1.3914018, -0.34089637]), "Wrong value on boxes"
assert np.isclose(classes[2].numpy(), 1.7079165), "Wrong value on classes"

print("\033[92m All tests passed!")

scores[2] = 8.147684
boxes[2] = [ 6.0797963   3.7433083   1.3914018  -0.34089637]
classes[2] = 1.7079165
scores.shape = (10,)
boxes.shape = (10, 4)
classes.shape = (10,)
 All tests passed!


**Ожидаемый вывод**:

<table>
    <tr>
        <td>
            <b>scores[2]</b>
        </td>
        <td>
           8.147684
        </td>
    </tr>
    <tr>
        <td>
            <b>boxes[2]</b>
        </td>
        <td>
           [ 6.0797963   3.743308    1.3914018  -0.34089637]
        </td>
    </tr>
    <tr>
        <td>
            <b>classes[2]</b>
        </td>
        <td>
           1.7079165
        </td>
    </tr>
        <tr>
        <td>
            <b>scores.shape</b>
        </td>
        <td>
           (10,)
        </td>
    </tr>
    <tr>
        <td>
            <b>boxes.shape</b>
        </td>
        <td>
           (10, 4)
        </td>
    </tr>
    <tr>
        <td>
            <b>classes.shape</b>
        </td>
        <td>
           (10,)
        </td>
    </tr>

</table>

### Обработка выхода нейронной сети

Пришло время реализовать функцию, принимающую выходные данные  CNN (мерное кодирование 19x19x5x85) и фильтрующую все рамки, используя ранее реализованные функции.


Implement `yolo_eval()` which takes the output of the YOLO encoding and filters the boxes using score threshold and NMS. There's just one last implementational detail you have to know. There're a few ways of representing boxes, such as via their corners or via their midpoint and height/width. YOLO converts between a few such formats at different times, using the following functions (which are provided): 

Реализуйте `yolo_eval()`, которая принимает выходные данные YOLO и фильтрует рамки, используя пороговую фильтрацию и NMS. 

Следует отметить один момент: существует несколько способов представления рамок, например, через их углы или через их середину и высоту/ширину. YOLO конвертирует несколько таких форматов между собой, используя следующие функции (которые даны):

```python
boxes = yolo_boxes_to_corners(box_xy, box_wh) 
```
которая конвертирует координаты рамок (x,y,w,h) в координаты углов (x1, y1, x2, y2), которые подаются в функцию `yolo_filter_boxes`
```python
boxes = scale_boxes(boxes, image_shape)
```

Сеть YOLO была обучена работать с изображениями размером 608x608. Если вы тестируете ее на изображении другого размера (например, в наборе данных для обнаружения автомобилей были изображения размером 720 x 1280), то происходит изменение масштаба полей так, чтобы их можно было разместить поверх исходного изображения размером 720 x 1280.


In [8]:
def yolo_boxes_to_corners(box_xy, box_wh):

    box_mins = box_xy - (box_wh / 2.)
    box_maxes = box_xy + (box_wh / 2.)

    return tf.keras.backend.concatenate([
        box_mins[..., 1:2],  # y_min
        box_mins[..., 0:1],  # x_min
        box_maxes[..., 1:2],  # y_max
        box_maxes[..., 0:1]  # x_max
    ])


In [9]:

def yolo_eval(yolo_outputs, image_shape = (720, 1280), max_boxes=10, score_threshold=.6, iou_threshold=.5):
    """
    
    Входные аргументы:
    yolo_outputs -- выход модели (для image_shape равного (608, 608, 3)) содержит 4 тензора:
                    box_xy: тензор размера (None, 19, 19, 5, 2)
                    box_wh:  тензор размера (None, 19, 19, 5, 2)
                    box_confidence:  тензор размера (None, 19, 19, 5, 1)
                    box_class_probs:  тензор размера (None, 19, 19, 5, 80)
    image_shape --  тензор размера (2,) содержит размер входа, здесь мы будем использовать (608., 608.)  (608. - float32)
    max_boxes -- целое число, максимальное количество рамок
    score_threshold -- пороговое значения CS
    iou_threshold -- пороговое значение "intersection over union" для NMS 
    
    Выходные значения:
    scores --  тензор размера (None, ), предсказанные CS для рамок
    boxes --  тензор размера (None, 4), предсказанные координаты рамок
    classes --  тензор размера (None,), предсказанные классы для рамок
    """
    
    # Распакуйте выходные значения нейронной сети
    box_xy, box_wh, box_confidence, box_class_probs = yolo_outputs
    
    # Сконвертируйте рамки для дальнейшей фильтрации [box_xy и box_wh -> координаты углов]
    boxes = yolo_boxes_to_corners(box_xy, box_wh)
    
    # Примените функцию, реализующую фильтрацию рамок по заданному score_threshold  (yolo_filter_boxes)
    scores, boxes, classes = yolo_filter_boxes(boxes, box_confidence, box_class_probs, threshold=score_threshold)
    
    # Отмасштабируйте рамки согласно оригинальному размеру изображения.
    boxes = scale_boxes(boxes, image_shape)
    
    # Премините MNS  c максимальным количеством рамок= max_boxes, и iou_threshold
    scores, boxes, classes = yolo_non_max_suppression(scores, boxes, classes, max_boxes=max_boxes, iou_threshold=iou_threshold)
    
    return scores, boxes, classes

In [14]:
tf.random.set_seed(10)
yolo_outputs = (tf.random.normal([19, 19, 5, 2], mean=1, stddev=4, seed = 1),
                tf.random.normal([19, 19, 5, 2], mean=1, stddev=4, seed = 1),
                tf.random.normal([19, 19, 5, 1], mean=1, stddev=4, seed = 1),
                tf.random.normal([19, 19, 5, 80], mean=1, stddev=4, seed = 1))
scores, boxes, classes = yolo_eval(yolo_outputs)
print("scores[2] = " + str(scores[2].numpy()))
print("boxes[2] = " + str(boxes[2].numpy()))
print("classes[2] = " + str(classes[2].numpy()))
print("scores.shape = " + str(scores.numpy().shape))
print("boxes.shape = " + str(boxes.numpy().shape))
print("classes.shape = " + str(classes.numpy().shape))

assert type(scores) == EagerTensor, "Use tensoflow functions"
assert type(boxes) == EagerTensor, "Use tensoflow functions"
assert type(classes) == EagerTensor, "Use tensoflow functions"

assert scores.shape == (10,), "Wrong shape"
assert boxes.shape == (10, 4), "Wrong shape"
assert classes.shape == (10,), "Wrong shape"
    
assert np.isclose(scores[2].numpy(), 171.60194), "Wrong value on scores"
assert np.allclose(boxes[2].numpy(), [-1240.3483, -3212.5881, -645.78, 2024.3052]), "Wrong value on boxes"
assert np.isclose(classes[2].numpy(), 16), "Wrong value on classes"
    
print("\033[92m All tests passed!")
# END UNIT TEST

scores[2] = 171.60194
boxes[2] = [-1240.3483 -3212.5881  -645.78    2024.3052]
classes[2] = 16
scores.shape = (10,)
boxes.shape = (10, 4)
classes.shape = (10,)
 All tests passed!


**Ожидаемый вывод**:

<table>
    <tr>
        <td>
            <b>scores[2]</b>
        </td>
        <td>
           171.60194
        </td>
    </tr>
    <tr>
        <td>
            <b>boxes[2]</b>
        </td>
        <td>
           [-1240.3483 -3212.5881  -645.78    2024.3052]
        </td>
    </tr>
    <tr>
        <td>
            <b>classes[2]</b>
        </td>
        <td>
           16
        </td>
    </tr> 
        <tr>
        <td>
            <b>scores.shape</b>
        </td>
        <td>
           (10,)
        </td>
    </tr>
    <tr>
        <td>
            <b>boxes.shape</b>
        </td>
        <td>
           (10, 4)
        </td>
    </tr>
    <tr>
        <td>
            <b>classes.shape</b>
        </td>
        <td>
           (10,)
        </td>
    </tr>

</table>